<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing<b> </font>

<font size="5"> <b>Lab 5: InSAR Processing using ESA's SNAP Toolbox <font color='rgba(200,0,0,0.2)'> -- [20 Points] </font> </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks</b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /><font color='rgba(200,0,0,0.2)'> <b>Due Date: </b> April 05, 2019 </font>
</font>

<font size="3"> This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. It is introducing you to the concepts of interferometric SAR (InSAR) processing using the European Space Agency's (ESA) Sentinel Application Platform (SNAP), a free software tool specifically designed to analyze data from all of ESA's Sentinel Sensors. In this lab, we specifically will be working with the Sentinel-1 constellation, a two-sensor C-band radar providing 10m-resolution SAR data globally every 6-12 days. As previously, all data analysis will be done in the framework of *Jupyter Notebooks*.  <br>

<b>In this chapter we introduce the following data analysis concepts:</b>

- What is special about the Sentinel-1 SAR acquisition mode
- How to read visualize and work with Sentinel-1 data within the SNAP toolbox.
- How to process InSAR data start-to-finish within SNAP.
</font>

<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES <u>THREE</u> HOMEWORK ASSIGNMENTS at its very end.</b></font> Complete all assignments to achieve full score. </font> <br>
<font size="3"> To submit your homework, please download your Jupyter Notebook from the server both asf PDF (*.pdf) and Notebook file (*.ipynb) and submit them as a ZIP bundle via Blackboard or email (to fjmeyer@alaska.edu). To download, please select the following options in the main menu of the notebook interface:

<ol type="1">
  <li><font color='rgba(200,0,0,0.2)'> <b> Save your notebook with all of its content</b></font> by selecting <i> File / Save and Checkpoint </i> </li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in Notebook format</b></font>, click on <i>File / Download as / Notebook (.ipynb)</i>  <font color='gray'>--- Downloading your file may take a bit as the Notebook will be about 100MB in size</font></li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in PDF format</b></font>, click on <i>File / Download as / PDF vs LaTeX (.pdf) </i></li>
</ol>

Contact me at fjmeyer@alaska.edu should you run into any problems.
</font>

</font>

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3"> The first step of this lab exercises on SAR image time series analysis is the import of necessary python libraries into your Jupyter Notebook. See the code cell below for information on which libraries are needed.
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    
</font>

In [1]:
import sys,os,re
import numpy as np
import datetime
import time
import glob
import csv
import gdal

# Setup plotting inside the notebook
%matplotlib inline
import matplotlib.pylab as plt

<font size="3"> We write a couple of helper functions that check that a given file or path exists. These provide simple error handling for some of our os library function calls:</font>

In [2]:
def fileExists(filename):
    if os.path.isfile(filename):
        return True
    else:
        print("The file " + filename + " is missing.")
        return False

In [3]:
def pathExists(path):
    if os.path.exists(path):
        return True
    else:
        print("Invalid Path.")
        return False

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Background on Analyzed Event </b> </font> <img src="NotebookAddons/Shakemap.jpg" width="350" align="right" /> 

<font size="3"> Interferometric SAR processing exploits the difference between the phase signals of repeated SAR acquisitions to analyze the shape and deformation of the Earth surface. The principles and concepts of Interferometric SAR (InSAR) processing were covered in Lectures 12 to 14 of <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657</a>. Please refer to information in the <a href="https://radar.community.uaf.edu/module-iii-advanced-radar-concepts/" target="_blank">lecture notes</a> should you have InSAR-related questions that are not explained in this document.

In the first part of this lab, we will analyze a pair of Sentinel-1 images bracketing the devastating 2016 Kumamoto earthquake, whose 6.5 magnitude foreshock and 7.0 main shock devastated large areas around Kumamoto, JP on April 14th and 16th, respectively. The figure to your right shows the USGS ShakeMap associated with the 7.0 magnitude main shock, showing both the violence of the event and the location of the largest devastation.

We will use ESA’s Sentinel Application Platform (SNAP) to perform InSAR processing on these Sentinel-1 images. The advantages of the SNAP tool include (1) its graphical user interface, which renders the SNAP tool straightforward to use (compared to other InSAR processing tools); (2) the easy-to-access, free-of-charge, and public domain nature of the SNAP tool; and (3) the fact that SNAP is an integrative multi-sensor toolbox and enables processing data from all Sentinel sensors within one joint processing platform. While the graphical user interface is not used here, it is available to you on your personal computer for future exercises.
</font>

<hr>
<font face="Calibri">

<font size="5"> <b> 2. Sentinel-1 Data used in this Lab </b> </font> <img src="NotebookAddons/Lab5-topsar.jpg" width="300" align="right" /> 

<font size="3"> The Interferometric Wide (IW) swath mode is the main acquisition mode over land for <a href="https://www.asf.alaska.edu/sentinel/" target="_blank">Sentinel-1</a>. It acquires data with a 250 km swath at 5 m by 20 m spatial resolution (single look). IW captures three sub-swaths using the Terrain Observation with Progressive Scans SAR (TOPSAR) acquisition principle. With the TOPSAR technique, in addition to steering the beam in range as in ScanSAR, the beam is also electronically steered from backward to forward in the azimuth direction for each burst, avoiding scalloping and resulting in homogeneous image quality throughout the swath. A schematic of the TOPSAR acquisition principle is shown to your right. For more information on TOPSAR please read <cite><a href="https://ieeexplore.ieee.org/abstract/document/1677745"><i>TOPSAR: Terrain Observation by Progressive Scans</i></a> by De Zan & Monti Guarnieri</cite>. 

IW Single Look Complex (SLC) products contain one image per sub-swath and one per polarization channel, for a total of three (single polarization) or six (dual polarization) images in an IW product. Each sub-swath image consists of a series of bursts, where each burst has been processed as a separate SLC image. The individually focused complex burst images are included, in azimuth-time order, into a single sub-swath image with black-fill demarcation in between.
<br><br>

<img src="NotebookAddons/Lab5-footprint.jpg" width="400" align="right" /> We will use a pair of repeated Sentine-1 images for this part of the lab that were acquired on April 8 and April 20, 2016, bracketing the fore- and main shock of the Kumamoto earthquake event. Hence, the phase difference between these image acquisitions capture the cumulative co-seismic deformation caused by both of these seismic events. The footprint of the Sentinel-1 images is shown to your right and covers the areas affected by the earthquake. Hence, Sentinel-1 data are a good basis for studying earthquake-related surface deformation.

We will download these images directly from the <b><a href="https://vertex.daac.asf.alaska.edu/#" target="_blank">ASF VERTEX</a></b> search client:
- <b>Pre-event image:</b> S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB
- <b>Post-event image:</b> S1A_IW_SLC__1SSV_20160420T091355_20160420T091423_010903_010569_F9CE

</font> 
</font>

<hr>
<font face="Calibri" size="3"> Before we get started, let's <b>first create a working directory for this analysis and change into it:</b> </font>

In [6]:
# Let's first create a working directory for this analysis and change into it
os.chdir('/home/jovyan/notebooks/ASF/GEOS_657_Labs')
!mkdir -p Kumamoto
os.chdir('/home/jovyan/notebooks/ASF/GEOS_657_Labs/Kumamoto')
os.getcwd() 

'/home/jovyan/notebooks/ASF/GEOS_657_Labs/Kumamoto'

<font face="Calibri" size="3"> Now, we <b>define a function to download our data sets from the ASF archives:</b> </font>

In [ ]:
import requests
from getpass import getpass
def download_file(url, username=None, password=None):
    #https://stackoverflow.com/questions/16694907/how-to-download-large-file-in-python-with-requests-py
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)   # NOTE the stream=True parameter
    if r.status_code == 401:
        pwd = getpass('Password for {}: '.format(username))
        r = requests.get(r.url, stream=True, auth=(username,pwd))
    print('Downloading...')
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    if os.stat(local_filename).st_size < 5000:
        print('Failed!  Check your password.')
    else:
        print('Done')
    return local_filename

<font face="Calibri" size="3"> You are ready to <b>authenticate yourself at ASF with your Earthdata login and download the files needed for this exercise.</b> To do so, replace the <i>NEDusr</i> "fmeyer" with your own username in the code cell below and run the cell: </font>

In [ ]:
from getpass import getpass

NEDusr = 'aflewandowski'
NEDpass = getpass()

with open('/home/jovyan/.netrc', 'w') as f:
    f.write('machine urs.earthdata.nasa.gov login ' + NEDusr + ' password ' + NEDpass + '\n')

In [ ]:
# Now, let's actually download the file:
url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip'
preevent = download_file(url, username=NEDusr, password=NEDpass)
url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SSV_20160420T091355_20160420T091423_010903_010569_F9CE.zip'
postevent = download_file(url, username=NEDusr, password=NEDpass)

<br>
<hr>
<font face="Calibri" size="5"> <b> 3. InSAR Processing using SNAP: </b> 

<font size="3"> We now will be going through the individual steps of the InSAR Processing flow using the <a href="http://step.esa.int/main/download/" target="_blank">SNAP toolbox</a>. To do so, <b>we first have to set up the path to the SNAP toolbox and define the names of the image files we want to process:</b></font>

</font> 

In [ ]:
baseSNAP = '/usr/local/snap/bin/gpt '

In [ ]:
prebaseGran = re.split('/',preevent.replace('.zip',''))[-1]
postbaseGran = re.split('/',postevent.replace('.zip',''))[-1]

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.1 Extract Sub-swath from the Data Files </b> </font>

<font face="Calibri" size="3"> SNAP natively processes the individual sub-swaths of a Sentinel-1 data set individually. Hence, we have to first extract the targeted sub-swath data from the data file. We will start with IW1, mostly because it is center nicely on the event: </font> 

In [ ]:
def topsarSplit(inData,ss,td2,subs):
    calFlag = 'TOPSAR-Split -Psubswath=%s -PselectedPolarisations=VV ' % ss
    if subs != '':
        calFlag = calFlag + '-PwktAoi=\"%s\" ' % subs
    out = '-t %s/%s ' % (td2,inData.replace('.zip','_TS'))
    inD = '-Ssource=%s' % inData
    cmd = baseSNAP + calFlag + out + inD
    print('Spliting subswaths')
    print(cmd)
    os.system(cmd)
    #This function uses .zip since it is the first step
    return '%s' % inData.replace('.zip','_TS.dim')

In [ ]:
msplit = topsarSplit(preevent,'IW1',os.getcwd(),'')

In [ ]:
ssplit = topsarSplit(postevent,'IW1',os.getcwd(),'')

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.2 Apply Orbit File </b> </font>

<font face="Calibri" size="3"> We are interested in measuring surface deformation for this analysis. Remember that the interferometric phase $\phi$ contains (at a minimum) the following information:

$\phi = \phi_{topo} + \phi_{defo} = \frac{4\cdot\pi}{\lambda} \frac{B_{\perp}}{R\cdot cos(\theta)} \cdot h + \frac{4\cdot\pi}{\lambda} \cdot \Delta R $
<br><br>
In order to be able to remove most of $\phi_{topo}$ from this equation, we need a good DEM as well as accurate information about the interferometric baseline $B_{\perp}$. To facilitate the latter, we update the orbit information in the data file to the post-processed <i>precise</i> orbits.

</font> 

In [ ]:
def applyOrbit(inData,td2):
    aoFlag = 'Apply-Orbit-File '
    oType = '-PcontinueOnFail=\"true\" -PorbitType=\'Sentinel Precise (Auto Download)\' '
    out = '-t %s/%s ' % (td2,inData.replace('.dim','_OB'))

    cmd = baseSNAP + aoFlag + out + oType + ' -Ssource=%s/%s' % (td2,inData)
    print('Applying Precise Orbit file')
    print(cmd)
    os.system(cmd)
    return '%s' % inData.replace('.dim','_OB.dim')

In [ ]:
mobOut = applyOrbit(msplit,os.getcwd())
print('\n\n')
sobOut = applyOrbit(ssplit,os.getcwd())
print('\n\n')
print(mobOut,sobOut)

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.3 Project DEM into SAR Imaging Geometry </b> </font>

<font face="Calibri" size="3"> The DEM is needed for a number of processing routines in the Sentinel-1 InSAR processing workflow. Steps where the DEM is involved include:

- The accurate co-registration of master and slave image
- The removal of the topographic phase component $\phi_{topo}$
- The geocoding and terrain correction of the final InSAR data

To facilitate all of these steps, <b>we need to retrieve a DEM from a public repository and project the DEM into the SAR observation geometry.</b>
</font> 

In [ ]:
def backGeocoding(inData1,inData2,td2,dem=''):
    calFlag = 'Back-Geocoding ' 
    if dem != '':
        calFlag = calFlag + '-PdemName=\"External DEM\" -PexternalDEMFile=%s ' % dem
    else: 
        calFlag = calFlag + '-PdemName=\"SRTM 1Sec HGT\" ' 

    out = '-t %s/%s ' % (td2,inData1.replace('.dim','_BG'))
    inD = '-SsourceProducts=\"%s/%s\" \"%s/%s\" ' % (td2,inData1,td2,inData2)
    cmd = baseSNAP + calFlag + out + inD
    print('Back Geocoding') 
    print(cmd) 
    os.system(cmd)
    return '%s' % inData1.replace('.dim','_BG.dim')

In [ ]:
# If orbit download doesn't work, do this ...
#mobOut=msplit
#sobOut=ssplit

In [ ]:
gcOut = backGeocoding(mobOut,sobOut,os.getcwd(),'')

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.4 Accurate Co-Registration using Enhanced Spectral Diversity (ESD) </b> </font>

<font face="Calibri" size="3"> For interferometric processing, two or more images must be co-registered into a common geometry. The geometry of one image is selected as the <i>master</i> geometry while the other images are often referred to as <i>slave</i> images. The pixels in slave images will be moved to align with the master image to sub-pixel accuracy.

Coregistration ensures that each ground target contributes to the same (range, azimuth) pixel in both the master and the slave image. For TOPS InSAR, co-registration using the Enhanced Spectral Diversity (ESD) method (published in <cite><a href="https://ieeexplore.ieee.org/abstract/document/6130599"><i>TOPS Interferometry With TerraSAR-X</i></a> by P. Prats-Iraola et al.</cite>) should be used. This approach can achieve co-registration qualities down to $\frac{1}{1000}$ of a pixel. 
</font> 

In [ ]:
def applyESD(inData,td2):
    calFlag = 'Enhanced-Spectral-Diversity ' 
    out = '%s/%s ' % (td2,inData.replace('.dim','_ESD'))
    inD = '-Ssource=%s/%s' % (td2,inData)
    cmd = baseSNAP + calFlag + '-t ' + out + inD
    print('Performing ESD') 
    print(cmd) 
    os.system(cmd)
    test = out.strip() + '.dim'
    if not os.path.isfile(test):
        print('\n*** Command Failed ***') 
        print('Error: Expected output file not found: '+test)
    return '%s' % inData.replace('.dim','_ESD.dim')

In [ ]:
esdOut = applyESD(gcOut,os.getcwd())

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.5 Interferogram Formation and Coherence Estimation </b> </font>

<font face="Calibri" size="3"> The interferogram is formed by cross-multiplying the master image with the complex conjugate of the slave. The amplitude of both images is multiplied while their respective phases are differenced to form the interferogram. 
The phase difference map, i.e., interferometric phase at each SAR image pixel depends only on the difference in the travel paths from each of the two SARs to the considered resolution cell.
    
Removal of the flat-Earth phase is done automatically during the interferogram formation step. The flat-Earth phase is the phase present in the interferometric signal due to the curvature of the reference surface. The flat-Earth phase is estimated using orbital and metadata information and subtracted from the complex interferogram.
</font> 

In [ ]:
def createInterferogram(inData,td2):
    calFlag = 'Interferogram ' 
    out = '%s/%s ' % (td2,inData.replace('.dim','_INT'))
    inD = '-SsourceProduct=%s/%s' % (td2,inData)
    cmd = baseSNAP + calFlag + '-t ' + out + inD
    print('Creating Interferogram ...') 
    print(cmd) 
    os.system(cmd)
    test = out.strip() + '.dim'
    if not os.path.isfile(test):
        print('\n*** Command Failed ***') 
        print('Error: Expected output file not found: '+test)
    return '%s' % inData.replace('.dim','_INT.dim')

In [ ]:
intOut = createInterferogram(esdOut,os.getcwd())

<br>
<hr>
<img src="NotebookAddons/Lab5-bursts.jpg" width="400" align="right" /><font face="Calibri" size="4"> <b> 3.6 TOPS Deburst </b> </font>

<font face="Calibri" size="3"> Within a subswath, TOPS data are acquired in bursts. Each burst is separated by demarcation zones. The image to your right demonstrates what the burst demarcations look like in the SAR amplitude image. Any ‘data’ within the demarcation zones can be considered invalid and should be zero-filled but may contain garbage values. 
    
Eventually, these demarcation zones between bursts need to be removed to seamlessly join all bursts in a swath into a single image. <b>We apply this step after interferogram formation using the TOPS Deburst operator.</b>
</font> 

In [ ]:
def topsarDeburst(inData,td2):
    calFlag = 'TOPSAR-Deburst ' 
    out = '%s/%s ' % (td2,inData.replace('.dim','_DB'))
    inD = '-Ssource=%s/%s' % (td2,inData)
    cmd = baseSNAP + calFlag + '-t ' + out + inD
    print('TOPSAR Deburst ...') 
    print(cmd) 
    os.system(cmd)
    test = out.strip() + '.dim'
    if not os.path.isfile(test):
        print('\n*** Command Failed ***') 
        print('Error: Expected output file not found: '+test)
    return '%s' % inData.replace('.dim','_DB.dim')

In [ ]:
dbOut = topsarDeburst(intOut,os.getcwd())

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.7 Remote Topographic Phase </b> </font>

<font face="Calibri" size="3"> To emphasize phase signatures related to deformation, topographic phase contributions are typically removed using a known DEM. In SNAP, the Topographic Phase Removal operator will simulate an interferogram based on a reference DEM and subtract it from the processed interferogram.
    
SNAP will automatically find and download the DEM segment required for correcting your interferogram of interest. After topographic phase removal, the resulting product will appear largely devoid of topographic influence. 
</font> 

In [ ]:
def topoPhaseRemoval(inData,td2,dem=''):
    calFlag = 'TopoPhaseRemoval ' 
    if dem != '':
        calFlag = calFlag + '-PdemName=\"External DEM\" -PexternalDEMFile=%s ' % dem
    else: 
        calFlag = calFlag + '-PdemName=\"SRTM 1Sec HGT\" ' 
    out = '%s/%s ' % (td2,inData.replace('.dim','_Topo'))
    inD = '-SsourceProduct=\"%s/%s\" ' % (td2,inData)
    cmd = baseSNAP + calFlag + '-t ' + out + inD
    print('Removing Topographic Phase') 
    print(cmd) 
    os.system(cmd)
    test = out.strip() + '.dim'
    if not os.path.isfile(test):
        print('\n*** Command Failed ***') 
        print('Error: Expected output file not found: '+test)
    return '%s' % inData.replace('.dim','_Topo.dim')

In [ ]:
topoOut = topoPhaseRemoval(dbOut,os.getcwd(),'')

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.8 Phase Filtering and Multi-looking </b> </font>

<img src="NotebookAddons/Lab5-filtered.jpg" width="400" align="right" /><font face="Calibri" size="3"> You will see in the GeoTIFF files you just created that up to this stage, your interferogram looks very nosy and fringe patterns are difficult to discern. Hence, we will apply two subsequent processing steps to reduce noise and enhance the appearance of the deformation fringes.

As discussed in Lecture 12, interferometric phase can be corrupted by noise from:

- Temporal decorrelation
- Geometric decorrelation
- Volume scattering
- Processing error

To be able to properly analyze the phase signatures in the interferogram, the signal-to-noise ratio will be increased by applying <b>phase filtering and multilooking</b>. Once these steps are concluded, your interferogram of the Kumamoto Earthquake should look like the example to the right.

<hr>
<b>In a first step</b> we use multilooking to improve phase fidelity. This process averages a user-specified number of range and azimuth pixels, resulting in a reduction of noise. We will use $12 \times 4$ range/azimuth looks, resulting in a pixel size of about 60m (Figure 9). 
</font> 

In [ ]:
def applyML(inData,td2):
    calFlag = 'Multilook -PnRgLooks=12 -PnAzLooks=4 '
    out = '%s/%s ' % (td2,inData.replace('.dim','_ML'))
    inD = '-Ssource=%s/%s' % (td2,inData)
    cmd = baseSNAP + calFlag + '-t ' + out + inD
    print('Applying Multilook') 
    print(cmd) 
    os.system(cmd)
    test = out.strip() + '.dim'
    if not os.path.isfile(test):
        print('\n*** Command Failed ***') 
        print('Error: Expected output file not found: '+test)
    return '%s' % inData.replace('.dim','_ML.dim')


In [ ]:
mlOut = applyML(topoOut,os.getcwd())

<hr>
<font face="Calibri" size="3">
<b>In a second step</b> we perform a phase filtering step using a state-of-the art filtering approach. Specifically, we will be using the Goldstein Phase Filtering published in <cite><a href="https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/1998GL900033%4010.1002/%28ISSN%291944-8007.GRL40"><i>Radar Interferogram Filtering for Geophysical Applications</i></a> by Goldstein & Werner</cite>.
</font> 

In [ ]:
def goldsteinFilter(inData,td2):
    calFlag = 'GoldsteinPhaseFiltering -PcoherenceThreshold=0.2 ' 
    out = '%s/%s ' % (td2,inData.replace('.dim','_GF'))
    inD = '-SsourceProduct=%s/%s' % (td2,inData)
    cmd = baseSNAP + calFlag + '-t ' + out + inD
    print('Goldstein Phase Filtering') 
    print(cmd) 
    os.system(cmd)
    test = out.strip() + '.dim'
    if not os.path.isfile(test):
        print('\n*** Command Failed ***') 
        print('Error: Expected output file not found: '+test)
    return '%s' % inData.replace('.dim','_GF.dim')

In [ ]:
gfOut = goldsteinFilter(mlOut,os.getcwd())

<br>
<hr>
<img src="NotebookAddons/Lab5-unwrapping.jpg" width="400" align="right" /><font face="Calibri" size="4"> <b> 3.9 Phase Unwrapping </b> </font>

<font face="Calibri" size="3"> The interferometric phase $\phi$ is originally only available in its wrapped form. Phase unwrapping tries to recover the unwrapped phase $\psi$ given its observed wrapped phase $\phi = W\{\psi\}$.
    
The solution for this problem is difficult to find and needs some assumptions. The most important of these assumptions is that we expect the true earth surface to be reasonably smooth, meaning that strong discontinuities as shown in the right image are rare. Phase unwrapping algorithms mathematically code the statement that the left image is a more likely solution for a measured vector field than the image to the right.<br><br>

We will use the public-domain phase unwrapping code SNAPHU (<u>s</u>tatistical-cost, <u>n</u>etwork-flow <u>ph</u>ase-<u>u</u>nwrapping algorithm) first published in <cite><a href="https://ieeexplore.ieee.org/abstract/document/1036000"><i>Phase Unwrapping for Large SAR Interferograms: Statistical Segmentation and Generalized Network Models</i></a> by Chen & Zebker</cite> to achieve this task. 

</font> 

In [ ]:
'''def phaseUnwrap(inData,td2):
    # First export to SNAPHU format
    calFlag = 'SnaphuExport ' 
    out = '-PtargetFolder=%s ' % td2
    inD = '-SsourceProduct=%s/%s' % (td2,inData)
    cmd = baseSNAP + calFlag + out + inD
    print('Exporting for Snaphu') 
    print(cmd) 
    os.system(cmd)

    # Actually call snaphu
    snaphuDir = '%s/%s' % (td2,inData.replace('.dim',''))
    snaphuConf = '%s/snaphu.conf' % snaphuDir
    han = open(snaphuConf,'r')
    sData = han.readlines()
    han.close()
    han = open(snaphuConf,'w')
    for i in range(0,len(sData)):
        if 'LOGFILE' in sData[i]:
            print 'Found logfile in snaphu.conf'
            sData[i] = sData[i].replace('LOGFILE','#LOGFILE')
        if i==6:
            snaphuCmd = sData[i].replace('#   ','')
        han.write(sData[i])
    han.close()
    cmd = 'cd %s;%s' % (snaphuDir,snaphuCmd)
    print(cmd) 
    os.system(cmd)

    # Import from SNAPHU format
    calFlag = 'SnaphuImport ' 
    out = '-t %s/%s ' % (td2,inData.replace('.dim','_UNW'))
    snaphuFile = glob.glob('%s/Unw*.hdr' % snaphuDir)[0]
    inD = '-SsourceProducts=\"%s/%s\" \"%s\" ' % (td2,inData,snaphuFile)
    cmd = baseSNAP + calFlag + out + inD
    print('Importing from Snaphu') 
    print(cmd) 
    os.system(cmd)
    return '%s' % inData.replace('.dim','_UNW.dim')'''

In [ ]:
#unwOut = phaseUnwrap(mlOut,os.getcwd())

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.10	Geocoding and Export in a User-defined Format </b> </font>

<font face="Calibri" size="3"> To make the data useful to geoscientists, the unwrapped interferometric phase image needs to be projected into a geographic coordinate system using a DEM-assisted geocoding step. 
    
In the following we first geocode the data set using the DEM. <b>We will geocode both the <i>unfiltered</i> and the <i>phase filtered</i> versions of the interferogram</b>. Subsequently, we convert the geocoded data sets into GeoTIFF format. Feel free to compare the filtered and unfiltered versions for their relative performance.

</font> 

In [ ]:
def applyTC(inData,td2,dem=''):
    calFlag = 'Terrain-Correction '
    out = '%s/%s  ' % (td2,inData.replace('.dim','_TC'))
    inD = '-Ssource=%s/%s' % (td2,inData)
    if dem != '':
        inD = inD + ' -PoutputComplex=true -PdemName=\"External DEM\" -PexternalDEMFile=%s -PexternalDEMNoDataValue=0 -PpixelSpacingInMeter=60 ' % extDEM
    else:
        #inD = inD + ' -PdemName=\"SRTM 3Sec\" '
        inD = inD + ' -PoutputComplex=true -PdemName=\"SRTM 1Sec HGT\" -PimgResamplingMethod=\"NEAREST_NEIGHBOUR\" '
    cmd = baseSNAP + calFlag + '-t ' + out + inD
    print('Applying Terrain Correction -- This will take some time') 
    print(cmd) 
    os.system(cmd)
    test = out.strip() + '.dim'
    if not os.path.isfile(test):
        print('\n*** Command Failed ***') 
        print('Error: Expected output file not found: '+test)
    return '%s' % inData.replace('.dim','_TC.dim')

In [ ]:
tcOut = applyTC(gfOut,os.getcwd(),'')

In [ ]:
tcUnfilteredOut = applyTC(mlOut,os.getcwd(),'')

In [ ]:
cmd = 'cd %s/%s*GF_TC.data;python /home/jovyan/notebooks/ASF/GEOS_657_Labs/makeSNAPIntf.py i*.img q*.img ' % (os.getcwd(),prebaseGran)
print(cmd) 
os.system(cmd)
cmd = 'cd %s/%s*Topo_ML_TC.data;python /home/jovyan/notebooks/ASF/GEOS_657_Labs/makeSNAPIntf.py i*.img q*.img ' % (os.getcwd(),prebaseGran)
print(cmd) 
os.system(cmd)

In [ ]:
cmd = 'cd %s/%s*GF_TC.data;gdal_translate -ot Byte -scale 0 1 0 255 coh*.img coh-%s.tif ' % (os.getcwd(),prebaseGran,re.split('/',os.getcwd())[-1])
print(cmd) 
os.system(cmd)
cmd = 'cd %s/%s*Topo_ML_TC.data;gdal_translate -ot Byte -scale 0 1 0 255 coh*.img coh-unfiltered-%s.tif ' % (os.getcwd(),prebaseGran,re.split('/',os.getcwd())[-1])
print(cmd) 
os.system(cmd)

In [ ]:
cmd = 'cd %s/%s*GF_TC.data;mv phase.tif phase-%s.tif ' % (os.getcwd(),prebaseGran,re.split('/',os.getcwd())[-1])
print(cmd) 
os.system(cmd)
cmd = 'cd %s/%s*Topo_ML_TC.data;mv phase.tif phase-unfiltered-%s.tif ' % (os.getcwd(),prebaseGran,re.split('/',os.getcwd())[-1])
print(cmd) 
os.system(cmd)

In [ ]:
cmd = 'cd %s/%s*GF_TC.data;mv amplitude.tif amp-%s.tif ' % (os.getcwd(),prebaseGran,re.split('/',os.getcwd())[-1])
print(cmd)
os.system(cmd)
cmd = 'cd %s/%s*Topo_ML_TC.data;mv amplitude.tif amplitude-unfiltered-%s.tif ' % (os.getcwd(),prebaseGran,re.split('/',os.getcwd())[-1])
print(cmd)
os.system(cmd)

In [ ]:
!mv */*.tif .
!rm amp-*.tif

<font face="Calibri" size="3"> At the very end, let's clean up some of the temporary hard drive space before we leave:
</font> 

In [ ]:
!rm *.zip
!rm -r *TS.d*
!rm -r *OB.d*
!rm -r *BG.d*
!rm -r *ESD.d*
!rm -r *INT.d*
!rm -r *DB.d*
!rm -r *Topo.d*
!rm -r *ML.d*
!rm -r *GF.d*

<img src="NotebookAddons/Lab5-geocoded.jpg" width="800" align="none" />

<br>
<hr>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #1</u>:  </font> Kumamoto Interferogram - Interpret Orientation of Motion & Rupture Zone </b> <font color='rgba(200,0,0,0.2)'> -- [6 Points] </font> </font>

<font face="Calibri" size="3"> Please download the file <i>phase-Kumamoto.tif</i> from the lab environment and visualize it either in QGIS or Google Earth. The interferometric phase shown in this file carries a wealth of information about surface deformation (strength and direction of motion) and the location of the surface rupture. The phase map is also a proxy for other earthquake-related parameters such as the energy released during an event.<br><br>

<u>Based on the phase patterns in the interferogram and considering the discussions we had in the lab, please discuss the following</u>:
<br><br>
<ol>
  <li>Add a line to your figure representing a likely location of the rupture zone of this earthquake. Discuss and justify your choice. <font color='rgba(200,0,0,0.2)'> -- [3 Points] </font></li>
    <br>
  <li>The dominant mechanism present at this earthquake is right-lateral strike slip with a component of normal faulting, resulting in a combination of horizontal slip along the fault and vertical uplift. Please discuss if you can separate these superimposed deformation signals (vertical and horizontal) using this interferogram alone? Justify your answer. If your answer is no, discuss a solution that may allow you to characterize the amount of vertical and horizontal motion. <font color='rgba(200,0,0,0.2)'> -- [3 Points] </font></li>
</ol>
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.1.A [2 Points]:</font> Insert figure of interferogram with rupture zone overlain </i> 

ADD FIGURE HERE!
</font>
</div>
<hr>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.1.B [1 Points]:</font> Discuss and justify your choice for rupture zone location </i> 

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<div class="alert alert-success"><font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 1.2 [3 Points]: </font> Can separate these superimposed deformation signals (vertical and horizontal) using this interferogram alone? </i> 

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #2</u>:  </font> Kumamoto Interferogram - Compare InSAR Data to ShakeMap Information</b> <font color='rgba(200,0,0,0.2)'> -- [4 Points] </font> </font>

<font face="Calibri" size="3"> The Earthquake Hazards Program of the U.S. Geological Survey is providing a wealth of information about all significant earthquakes around the globe. The ShakeMap® is one of many sets of information included in this USGS feed. It was developed by the USGS to facilitate communication of earthquake information beyond just magnitude and location. By rapidly mapping out earthquake ground motions, ShakeMap portrays the distribution and severity of shaking. <br><br>

To access the ShakeMap for the Kumamoto event please visit <a href="https://earthquake.usgs.gov/earthquakes/eventpage/us20005iis/executive#executive" target="_blank">earthquake.usgs.gov/earthquakes/eventpage/us20005iis/executive#executive</a> and download the Event
KML (see below where to find the KML download on the page).
<img src="NotebookAddons/Lab5-WhereToFindShakemapkml.jpg" width="800" align="none" />
<br><br>

<u>For this assignment, please complete the following two tasks</u>:
<br>
<ol>
    <li>Overlay the ShakeMap onto your interferogram <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li><br>
    <li>Provide a discussion on how well the map conforms with the interferogram. Where do the two sets of information match up? Are there places where they don’t match up? <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
</ol>
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 2.1 [2 Points]:</font> Insert figure showing overlay of ShakeMap on the Kumamoto Interferogram </i> 

ADD FIGURE HERE!
</font>
</div>
<hr>

<div class="alert alert-success"><font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 2.2 [2 Points]: </font> Discussion on how well the map conforms with the interferogram. </i> 

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #3</u>:  </font> Rerun Complete Processing Flow to Create Interferogram for the 2018 M7.2 Oaxaca, Mexico Earthquake </b> <font color='rgba(200,0,0,0.2)'> -- [10 Points] </font> </font>

<font face="Calibri" size="3"> The 2018 Oaxaca earthquake occurred on February 16, 2018 at 17:39 local time (23:39 UTC) in the Sierra Madre del Sur in Oaxaca state in Southern Mexico. It had a magnitude of 7.2 on the moment magnitude scale and a maximum felt intensity of VII (very strong) on the Mercalli intensity scale. The hypocenter was located at a depth of 24.6 km and approximately 37 km northeast of Pinotepa de Don Luis. There were only two reports of injuries from the earthquake, but later a military helicopter surveying the damage crashed and killed 14 people. There were also 15 people injured.<br><br>

The depth and focal mechanism solutions of the event are consistent with its occurrence on the subduction zone interface between these plates, approximately 90 km northeast of the Middle America Trench, where the Cocos plate begins its descent into the mantle beneath Mexico. In the region of this earthquake, the Cocos plate moves approximately northeastward at a rate of 60 mm/yr.
    
To access the ShakeMap for the Oaxaca event please visit <a href="https://earthquake.usgs.gov/earthquakes/eventpage/us20005iis/executive#executive" target="_blank">earthquake.usgs.gov/earthquakes/eventpage/us20005iis/executive#executive</a> and download the Event
KML.<br>

<u>For this assignment, please complete the following two tasks</u>:
<br>
<ol>
    <li>Rerun the processing flow from start to finish using the following two image frames as your pre-event and post-event data sets: <font color='rgba(200,0,0,0.2)'> -- [6 Points] </font>
    <ul>
        <li> <b>Pre-event image:</b> S1A_IW_SLC__1SDV_20180127T122625_20180127T122652_020340_022BE7_3F5E</li>
        <li> <b>Post-event image:</b> S1A_IW_SLC__1SDV_20180220T122625_20180220T122651_020690_023710_6CEF</li>
    </ul>
    </li><br>
    <li>Access the ShakeMap for the 2018 M7.2 Oaxaca, Mexico Earthquake via <a href="https://earthquake.usgs.gov/earthquakes/eventpage/us2000d3km/executive#executive" target="_blank">earthquake.usgs.gov/earthquakes/eventpage/us2000d3km/executive#executive</a>. Dowload the Event KML and overlay it on your interferogram <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li><br>
<li>Discuss how the ShakeMap information compares to the InSAR data for this event. <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
</ol>
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 3.1 [6 Points]:</font> Rerun the InSAR processing flow for the 2018 Oaxaca, Mexico event. </i> 

To do so, replace the 2nd and 5th code cell in this notebook with the the code-cells shown below. Then run through the whole workflow:
</font>
</div>
<hr>

In [ ]:
# Replace 2nd code cell with this one
os.chdir('/home/jovyan/notebooks/ASF/GEOS_657_Labs')
!mkdir -p Oaxaca
os.chdir('/home/jovyan/notebooks/ASF/GEOS_657_Labs/Oaxaca')
os.getcwd() 

In [ ]:
# Replace 5th code cell with this one to downloading files for Oaxaca Earthquake:
url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20180127T122625_20180127T122652_020340_022BE7_3F5E.zip'
preevent = download_file(url, username=NEDusr, password=NEDpass)
url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20180220T122625_20180220T122651_020690_023710_6CEF.zip'
postevent = download_file(url, username=NEDusr, password=NEDpass)

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 3.2 [2 Points]:</font> Download the ShakeMap and create a figure with the ShakeMap overlain on your interferogram
    
ADD FIGURE HERE!
</font>
</div>
<hr>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Question 3.3 [2 Points]:</font> Discuss how the ShakeMap information compares to the InSAR data for this event.
    
PROVIDE YOUR DISCUSSION HERE!
</font>
</div>
<hr>

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - Feb 2019 </i>
</font>